# INSTALL JULIA - COLAB

PEGUE O SCRIPT DE TEMPLATE [AQUI](https://colab.research.google.com/drive/1lLHnAUl1kqPB6Fc1tF9r19om24DbYYFh)!

In [ ]:
# %%shell
# set -e

# #---------------------------------------------------#
# JULIA_VERSION="1.10.4" # any version ≥ 0.7.0
# JULIA_PACKAGES="IJulia BenchmarkTools Plots"
# JULIA_NUM_THREADS=2
# #---------------------------------------------------#

# if [ -z `which julia` ]; then
# # Install Julia
# JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
# echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
# BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
# URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
# wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
# tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
# rm /tmp/julia.tar.gz

# # Install Packages
# for PKG in `echo $JULIA_PACKAGES`; do
#     echo "Installing Julia package $PKG..."
#     julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
# done

# # Install kernel and rename it to "julia"
# echo "Installing IJulia kernel..."
# julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
#     "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
# KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
# KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
# mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

# echo ''
# echo "Successfully installed `julia -v`!"
# echo "Please reload this page (press Ctrl+R or F5) then"
# echo "jump to the 'Checking the Installation' section."
# fi

In [ ]:
versioninfo()

# PROJETO 2

## LIBS

In [ ]:
using Statistics, DataStructures, Plots

## VARS & VECS

In [ ]:
# Inicializando Lista de vetores e tempos
array_size = [10^i for i in 4:8]
array_list = [rand(1:10^3, i) for i in array_size]

bubblesort_time = zeros(length(array_size))
heapsort_time = zeros(length(array_size))

## Utils

In [ ]:
function split_subarrays(array)
    n = length(array)
    sqrt_n = floor(Int, sqrt(n))
    subarrays = [array[i:min(i+sqrt_n-1, n)] for i in 1:sqrt_n:n]
    return subarrays
end

In [ ]:
function my_benchmark(dummy_func, dummy_time)
    for i in 1:length(array_list)
        median_time = zeros(benchmark_count)
        for c in 1:benchmark_count
            array_copy = copy(array_list[i])
            start = time_ns()
            dummy_func(array_copy)
            final = time_ns()
            median_time[c] = (final - start) / 1e9
        end
        dummy_time[i] = median(median_time)
    end
end

## Heap

In [ ]:
function sqrtsort_heap(array)
    n = length(array)
    subarrays = split_subarrays(array)
    
    # Cria uma MaxHeap com cada subvetor
    heaps = [BinaryMaxHeap(sub) for sub in subarrays]
    
    solution = Vector{Int}(undef, n)
    current_pos = n
    
    while !isempty(heaps)
        # Encontrar o maior elemento em cada heap
        max_elements = [first(heap) for heap in heaps]
        max_index = argmax(max_elements)
        
        # Inserir o maior elemento na posição correta no vetor solução
        solution[current_pos] = pop!(heaps[max_index])
        current_pos -= 1
        
        if isempty(heaps[max_index])
            deleteat!(heaps, max_index)
        end
    end
    
    # print("Heap $n: $solution\n\n")
    return solution
end

## BubbleSort

In [ ]:
function bubblesort!(sub)
    n = length(sub)
    for i in 1:n-1
        for j in 1:n-i
            if sub[j] > sub[j+1]
                sub[j], sub[j+1] = sub[j+1], sub[j]
            end
        end
    end
end

In [ ]:
function sqrtsort_bubble(array)
    n = length(array)
    subarrays = split_subarrays(array)
    
    # Ordenar cada subvetor usando Bubble Sort
    [bubblesort!(sub) for sub in subarrays]

    solution = Vector{Int}(undef, n)
    current_pos = n
    
    while !isempty(subarrays)
        # Encontrar o maior elemento em cada subvetor
        max_elements = [last(sub) for sub in subarrays]
        max_index = argmax(max_elements)
        
        # Inserir o maior elemento na posição correta no vetor solução
        solution[current_pos] = pop!(subarrays[max_index])
        current_pos -= 1       
        
        if isempty(subarrays[max_index])
            deleteat!(subarrays, max_index)
        end
    end
    
    # print("Bubble $n: $solution\n\n")
    return solution
end

## Plot

In [ ]:
function generate_plot(bubblesort_time, heapsort_time, array_size)
    p = plot(
        title = "BubbleSort vs HeapSort",
        xlabel = "Array Size", 
        ylabel = "Time (seconds)", 
        xscale = :log10, 
        yscale = :log10,
        legend=:topleft,
    )

    plot!(
        p,
        array_size, bubblesort_time,
        label = "BubbleSort",
        lw = 2,
        linecolor = :blue  # Distinguindo as cores
    )

    
    plot!(
        p,
        array_size, heapsort_time,
        label = "HeapSort", 
        lw = 2,
        linecolor = :red  # Distinguindo as cores
    )

    display(p)
    savefig(p, "Projeto 2/imgs/plot2.png")
end

## Execução

In [ ]:
benchmark_count = 1

In [12]:
print("Bubble: ")
start = time_ns()
my_benchmark(sqrtsort_bubble, bubblesort_time)
print((time_ns() - start) / 1e9)

In [ ]:
print("\nHeap: ")
start2 = time_ns()
my_benchmark(sqrtsort_heap, heapsort_time)
print((time_ns() - start2) / 1e9)
println("\nTotal: ", (time_ns() - start) / 1e9)

In [ ]:
# Geração do gráfico de comparação
generate_plot(bubblesort_time, heapsort_time, array_size)